In [339]:
import pandas as pd
import numpy as np

In [340]:
# Read data from files 

# Load the dataset
file_path = 'labeled_lyrics_cleaned.csv'
lyrics_data = pd.read_csv(file_path)

file_path = 'SpotifyFeatures.csv'
track_data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
lyrics_data.head()



,col_0,artist,seq,song,label
0,0,Elijah Blake,"No, no\nI ain't ever trapped out the bando\nBu...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\nShe fo...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [341]:
track_data.head()

lyrics_merge = pd.merge(lyrics_data, track_data, how='inner', left_on=['artist', 'song'], right_on=['artist_name', 'track_name'])

lyrics_merge.head()



,col_0,artist,seq,song,label,genre,artist_name,track_name,track_id,popularity,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,108,Elizabeth Mitchell,"Little bird, little bird\nFly through my windo...","Little Bird, Little Bird",0.641,Folk,Elizabeth Mitchell,"Little Bird, Little Bird",5mdDzJBkQ7Gz7ePDCB397w,41,...,0.223,0.007110,C,0.121,-17.700,Minor,0.3670,175.357,4/4,0.641
1,109,Elizabeth Mitchell,"I've got a friend in Baltimore, little Liza Ja...",Little Liza Jane,0.701,Children's Music,Elizabeth Mitchell,Little Liza Jane,1S8HTkZ8KmXM5wCFIGtJ42,5,...,0.227,0.000000,B,0.118,-14.411,Minor,0.1850,105.772,4/4,0.738
2,111,Elizabeth Mitchell,Who's my pretty baby\nWho's my pretty little b...,Who's My Pretty Baby,0.664,Folk,Elizabeth Mitchell,Who's My Pretty Baby,2mZxNdZTN7Uuhv5W4IwzC8,39,...,0.257,0.072400,E,0.333,-15.539,Minor,0.0312,107.803,4/4,0.664
3,116,Elizabeth Mitchell,"Go to sleep, go to sleep\nGo to sleep you litt...",Sleep Eye,0.793,Folk,Elizabeth Mitchell,Sleep Eye,14FcnjzhhpjLBg8UnjbtGp,39,...,0.211,0.473000,D,0.073,-17.952,Minor,0.4920,176.193,4/4,0.793
4,165,Ella Eyre,"Not a word, from your lips\nYou just took for ...",We Don't Have To Take Our Clothes Off,0.332,Pop,Ella Eyre,We Don't Have To Take Our Clothes Off,48wSJ6NhMFYYB6LjdnPNjM,65,...,0.231,0.000004,C,0.111,-6.316,Minor,0.0334,96.729,3/4,0.332


In [342]:
# remove unwanted columns
lyrics_merge.drop(columns=['artist', 'song', 'col_0', 'label'], inplace=True)

In [343]:
lyrics_merge['seq'] = lyrics_merge['seq'].replace('¶', ' ', regex=True)

# Add sentence length
lyrics_merge['lyrics_length'] = lyrics_merge['seq'].apply(lambda x: len(x.split()))

# Drop lyrics column
lyrics_merge.drop(columns=['seq'], inplace=True)

lyrics_merge.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,lyrics_length
0,Folk,Elizabeth Mitchell,"Little Bird, Little Bird",5mdDzJBkQ7Gz7ePDCB397w,41,0.960,0.487,148213,0.223,0.007110,C,0.121,-17.700,Minor,0.3670,175.357,4/4,0.641,192
1,Children's Music,Elizabeth Mitchell,Little Liza Jane,1S8HTkZ8KmXM5wCFIGtJ42,5,0.789,0.855,145947,0.227,0.000000,B,0.118,-14.411,Minor,0.1850,105.772,4/4,0.738,218
2,Folk,Elizabeth Mitchell,Who's My Pretty Baby,2mZxNdZTN7Uuhv5W4IwzC8,39,0.899,0.728,113013,0.257,0.072400,E,0.333,-15.539,Minor,0.0312,107.803,4/4,0.664,116
3,Folk,Elizabeth Mitchell,Sleep Eye,14FcnjzhhpjLBg8UnjbtGp,39,0.960,0.595,139373,0.211,0.473000,D,0.073,-17.952,Minor,0.4920,176.193,4/4,0.793,183
4,Pop,Ella Eyre,We Don't Have To Take Our Clothes Off,48wSJ6NhMFYYB6LjdnPNjM,65,0.953,0.572,230711,0.231,0.000004,C,0.111,-6.316,Minor,0.0334,96.729,3/4,0.332,290


In [344]:
# Function to convert pitch to number
def pitch_to_number(pitch):
    pitch_map = {
        'C': 0,
        'C#': 1, 'Db': 1,
        'D': 2,
        'D#': 3, 'Eb': 3,
        'E': 4, 'Fb': 4,
        'E#': 5, 'F': 5,
        'F#': 6, 'Gb': 6,
        'G': 7,
        'G#': 8, 'Ab': 8,
        'A': 9,
        'A#': 10, 'Bb': 10,
        'B': 11, 'Cb': 11
    }
    return pitch_map.get(pitch, None)

# Replace the pitch column with its numeric representation
# Assuming the column name in your DataFrame that contains pitch values is 'pitch_column'
lyrics_merge['key'] = lyrics_merge['key'].apply(pitch_to_number)


In [345]:
# Convert the mode column to 1 for major and 0 for minor
lyrics_merge['mode'] = lyrics_merge['mode'].replace("Minor", 0)
lyrics_merge['mode'] = lyrics_merge['mode'].replace("Major", 1)

lyrics_merge.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,lyrics_length
0,Folk,Elizabeth Mitchell,"Little Bird, Little Bird",5mdDzJBkQ7Gz7ePDCB397w,41,0.960,0.487,148213,0.223,0.007110,0,0.121,-17.700,0,0.3670,175.357,4/4,0.641,192
1,Children's Music,Elizabeth Mitchell,Little Liza Jane,1S8HTkZ8KmXM5wCFIGtJ42,5,0.789,0.855,145947,0.227,0.000000,11,0.118,-14.411,0,0.1850,105.772,4/4,0.738,218
2,Folk,Elizabeth Mitchell,Who's My Pretty Baby,2mZxNdZTN7Uuhv5W4IwzC8,39,0.899,0.728,113013,0.257,0.072400,4,0.333,-15.539,0,0.0312,107.803,4/4,0.664,116
3,Folk,Elizabeth Mitchell,Sleep Eye,14FcnjzhhpjLBg8UnjbtGp,39,0.960,0.595,139373,0.211,0.473000,2,0.073,-17.952,0,0.4920,176.193,4/4,0.793,183
4,Pop,Ella Eyre,We Don't Have To Take Our Clothes Off,48wSJ6NhMFYYB6LjdnPNjM,65,0.953,0.572,230711,0.231,0.000004,0,0.111,-6.316,0,0.0334,96.729,3/4,0.332,290


In [346]:
# Handle outliers in the dataset
# remove song that are to long in duration songs longer than 500000 ms (500 seconds = 8.3 minutes)

lyrics_merge.head()
lyrics_merge = lyrics_merge[lyrics_merge['duration_ms'] < 500000]


# Remove unwanted columns
lyrics_merge.drop(columns=['artist_name', 'track_name', 'track_id', 'time_signature'], inplace=True)


In [347]:
from sklearn.preprocessing import MinMaxScaler


numerical_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 
                  'instrumentalness', 'liveness', 'loudness', 'speechiness', 
                  'tempo', 'valence', 'lyrics_length']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Scale the numerical columns using MinMaxScaler
lyrics_merge[numerical_cols] = scaler.fit_transform(lyrics_merge[numerical_cols])

lyrics_merge.head()

,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,lyrics_length
0,Folk,41,0.964824,0.467722,0.251087,0.217576,0.007263,0,0.104585,0.432811,0,0.369572,0.705491,0.637810,0.130912
1,Children's Music,5,0.792965,0.872429,0.246238,0.221609,0.000000,11,0.101515,0.535332,0,0.174440,0.348045,0.739349,0.148732
2,Folk,39,0.903517,0.732761,0.175754,0.251858,0.073953,4,0.321531,0.500171,0,0.009542,0.358478,0.661886,0.078821
3,Folk,39,0.964824,0.586495,0.232168,0.205477,0.483146,2,0.055465,0.424956,0,0.503592,0.709785,0.796922,0.124743
4,Pop,65,0.957789,0.561201,0.427646,0.225643,0.000004,0,0.094351,0.787662,0,0.011901,0.301593,0.314352,0.198081


In [348]:

# remove the rows with genre with less than 100 songs
lyrics_merge = lyrics_merge.groupby('genre').filter(lambda x: len(x) > 400)

print(lyrics_merge['genre'].value_counts())

# Make a datafram for each genre
country = lyrics_merge[lyrics_merge['genre'] == 'Country']
alternative = lyrics_merge[lyrics_merge['genre'] == 'Alternative']
rock = lyrics_merge[lyrics_merge['genre'] == 'Rock']
children = lyrics_merge[lyrics_merge['genre'] == 'Children’s Music']
folk = lyrics_merge[lyrics_merge['genre'] == 'Folk']
dance = lyrics_merge[lyrics_merge['genre'] == 'Dance']
blues = lyrics_merge[lyrics_merge['genre'] == 'Blues']
pop = lyrics_merge[lyrics_merge['genre'] == 'Pop']
rnb = lyrics_merge[lyrics_merge['genre'] == 'R&B']
indie = lyrics_merge[lyrics_merge['genre'] == 'Indie']
soul = lyrics_merge[lyrics_merge['genre'] == 'Soul']
rap = lyrics_merge[lyrics_merge['genre'] == 'Rap']
ska = lyrics_merge[lyrics_merge['genre'] == 'Ska']
hiphop = lyrics_merge[lyrics_merge['genre'] == 'Hip-Hop']
jazz = lyrics_merge[lyrics_merge['genre'] == 'Jazz']

list_of_genres = [country, rock, children, folk, dance, blues, rnb, indie, soul, rap, ska, hiphop, alternative, jazz]


genre
Country             1280
Alternative         1274
Rock                1243
Children’s Music    1198
Folk                1115
Dance                944
Blues                922
Pop                  825
R&B                  777
Indie                766
Soul                 725
Rap                  690
Ska                  634
Hip-Hop              532
Jazz                 406
Name: count, dtype: int64


In [349]:

for genre in list_of_genres:
    print(genre["genre"].iloc[0])
    genre.drop(columns=['genre'], inplace=True)
    print(genre.head())

dict_of_genres = {
    'country': country,
    'Alternative': alternative,
    'rock': rock,
    'children': children,
    'folk': folk,
    'dance': dance,
    'blues': blues,
    'rnb': rnb,
    'indie': indie,
    'soul': soul,
    'rap': rap,
    'ska': ska,
    'hiphop': hiphop,
    'Jazz': jazz
}

Country
     popularity  acousticness  danceability  duration_ms    energy  \
132          44      0.056682      0.537006     0.574537  0.564424   
250          48      0.123617      0.644782     0.292065  0.650128   
265          46      0.123617      0.350049     0.454517  0.593664   
268          73      0.272361      0.502914     0.361806  0.747931   
269          42      0.037386      0.455625     0.445186  0.632987   

     instrumentalness  key  liveness  loudness  mode  speechiness     tempo  \
132          0.000052    3  0.088211  0.758798     1     0.010614  0.605914   
250          0.000398    9  0.100491  0.730308     0     0.063043  0.616578   
265          0.000001    7  0.115841  0.876407     1     0.009864  0.495323   
268          0.000000    6  0.102538  0.881113     0     0.019513  0.564475   
269          0.000000    6  0.103561  0.834170     1     0.005361  0.595095   

      valence  lyrics_length  
132  0.099759       0.222755  
250  0.452528       0.116518  
265

/tmp/ipykernel_125909/521243162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre.drop(columns=['genre'], inplace=True)
/tmp/ipykernel_125909/521243162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre.drop(columns=['genre'], inplace=True)
/tmp/ipykernel_125909/521243162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre.drop(columns=['genre'], inplace=True)
/tmp/ipykernel_125909/521243162.py:3: SettingWithCopyWa

In [350]:

# See flops and bops for each genre
for genre in dict_of_genres.values():
    popularity_threshold = genre['popularity'].median()
    print(popularity_threshold)

    genre['flop_or_bop'] = np.where(
        genre['popularity'] < popularity_threshold, "flop", "bop"
    )

    # turn flop_or_bop into a int value for model training
    genre['flop_or_bop'] = np.where(
        genre['flop_or_bop'] == "flop", 0, 1
    )

    print(genre['flop_or_bop'].value_counts())

    genre.drop(columns=['popularity'], inplace=True)

45.0
flop_or_bop
1    682
0    598
Name: count, dtype: int64
50.0
flop_or_bop
1    643
0    631
Name: count, dtype: int64
59.0
flop_or_bop
1    691
0    552
Name: count, dtype: int64
54.5
flop_or_bop
1    599
0    599
Name: count, dtype: int64
48.0
flop_or_bop
1    616
0    499
Name: count, dtype: int64
56.0
flop_or_bop
1    478
0    466
Name: count, dtype: int64
35.0
flop_or_bop
1    479
0    443
Name: count, dtype: int64
52.0
flop_or_bop
1    391
0    386
Name: count, dtype: int64
54.0
flop_or_bop
1    384
0    382
Name: count, dtype: int64
46.0
flop_or_bop
1    365
0    360
Name: count, dtype: int64
60.0
flop_or_bop
1    354
0    336
Name: count, dtype: int64
28.0
flop_or_bop
1    334
0    300
Name: count, dtype: int64
56.0
flop_or_bop
1    292
0    240
Name: count, dtype: int64
44.0
flop_or_bop
1    209
0    197
Name: count, dtype: int64


/tmp/ipykernel_125909/4212895266.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['flop_or_bop'] = np.where(
/tmp/ipykernel_125909/4212895266.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['flop_or_bop'] = np.where(
/tmp/ipykernel_125909/4212895266.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ge

In [351]:
# split
from sklearn.model_selection import train_test_split
# models
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


# Split the data into training and test sets
for genre_name, genre in dict_of_genres.items():
    print(genre_name)
    X_train, X_test, y_train, y_test = train_test_split(
    genre.drop(columns=['flop_or_bop']), 
    genre['flop_or_bop'], 
    test_size=0.2, 
    random_state=1337,
    stratify=genre['flop_or_bop']
)
    print("starting")
    print(X_train.value_counts())
    print(y_train.value_counts())
    print("Starting training for a genre...")
    learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
    for learning_rate in learning_rates:
        gb = GradientBoostingClassifier(n_estimators=1,
                                        learning_rate = learning_rate,
                                        max_depth = 3, 
                                        random_state = 1337)
        gb.fit(X_train, y_train)
        print("Learning rate: ", learning_rate)
        print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
        print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test, y_test)))
    input("Press Enter to continue...")
    
    
  

country
starting
acousticness  danceability  duration_ms  energy    instrumentalness  key  liveness  loudness  mode  speechiness  tempo     valence   lyrics_length
0.865326      0.399538      0.488788     0.367810  0.015117          0    0.055772  0.644151  1     0.008256     0.212865  0.272480  0.094585         1
              0.418234      0.433058     0.377892  0.000051          0    0.096398  0.684486  1     0.013080     0.317348  0.359364  0.200822         1
              0.588695      0.208370     0.341594  0.003156          7    0.113795  0.589321  1     0.018870     0.414475  0.489166  0.096642         1
0.866332      0.387441      0.238646     0.082366  0.000000          7    0.075829  0.350924  1     0.012008     0.237316  0.540458  0.061686         1
              0.442428      0.333954     0.272023  0.000004          2    0.918133  0.632056  1     0.348129     0.203326  0.383440  0.255655         1
                                                                            